In [271]:
import pandas as pd
import re

In [272]:
movies = pd.read_csv("movies_metadata.csv")
keywords = pd.read_csv("keywords.csv")
credits = pd.read_csv("credits.csv")

/var/folders/5s/7snm3zrn1j134tv9rl5h0vf40000gn/T/ipykernel_1219/2597897387.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("movies_metadata.csv")


### Parsing

#### Movies parsing

In [273]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [274]:
movies = movies[["adult", "genres", "id", "original_language", "overview", "popularity", "production_companies", "release_date", "title", "vote_average"]]
movies.head()

,adult,genres,id,original_language,overview,popularity,production_companies,release_date,title,vote_average
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,Toy Story,7.7
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15,Jumanji,6.9
2,False,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995-12-22,Grumpier Old Men,6.5
3,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,1995-12-22,Waiting to Exhale,6.1
4,False,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...",1995-02-10,Father of the Bride Part II,5.7


In [275]:
movies.isna().sum()

adult                     0
genres                    0
id                        0
original_language        11
overview                954
popularity                5
production_companies      3
release_date             87
title                     6
vote_average              6
dtype: int64

In [276]:
movies = movies.dropna(subset=["original_language", "title", "popularity", "production_companies", "release_date", "vote_average"])
movies = movies.fillna("")
movies.isna().sum()

adult                   0
genres                  0
id                      0
original_language       0
overview                0
popularity              0
production_companies    0
release_date            0
title                   0
vote_average            0
dtype: int64

In [277]:
movies = movies.astype({"adult": bool,
                        "id": int,
                        "original_language": str,
                        "overview": str,
                        "popularity": float,
                        "title": str,
                        "vote_average": float})
movies["genres"] = movies["genres"].map(eval)
movies["production_companies"] = movies["production_companies"].map(eval)
movies["release_date"] = pd.to_datetime(movies["release_date"])
movies.dtypes

adult                             bool
genres                          object
id                               int64
original_language               object
overview                        object
popularity                     float64
production_companies            object
release_date            datetime64[ns]
title                           object
vote_average                   float64
dtype: object

In [278]:
movies.index = movies["id"]
movies = movies.drop(columns=["id"])
movies = movies.sort_index()
movies.head()

,adult,genres,original_language,overview,popularity,production_companies,release_date,title,vote_average
id,,,,,,,,,
2,True,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",fi,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,"[{'name': 'Villealfa Filmproduction Oy', 'id':...",1988-10-21,Ariel,7.1
3,True,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",fi,"An episode in the life of Nikander, a garbage ...",2.292110,"[{'name': 'Villealfa Filmproduction Oy', 'id':...",1986-10-16,Shadows in Paradise,7.1
5,True,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",en,It's Ted the Bellhop's first night on the job....,9.026586,"[{'name': 'Miramax Films', 'id': 14}, {'name':...",1995-12-09,Four Rooms,6.5
6,True,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",en,"While racing to a boxing match, Frank, Mike, J...",5.538671,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",1993-10-15,Judgment Night,6.4
11,True,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",en,Princess Leia is captured and held hostage by ...,42.149697,"[{'name': 'Lucasfilm', 'id': 1}, {'name': 'Twe...",1977-05-25,Star Wars,8.1


#### Keywords parsing

In [279]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [280]:
keywords.isna().sum()

id          0
keywords    0
dtype: int64

In [281]:
keywords["keywords"] = keywords["keywords"].map(eval)

In [282]:
keywords.index = pd.to_numeric(keywords["id"])
keywords = keywords.drop(columns=["id"])
keywords

,keywords
id,
862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...
439050,"[{'id': 10703, 'name': 'tragic love'}]"
111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
67758,[]


#### Credits parsing

In [283]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [284]:
credits.isna().sum()

cast    0
crew    0
id      0
dtype: int64

In [285]:
credits.index = pd.to_numeric(credits["id"])
credits = credits.drop(columns=["id"])
credits["cast"] = credits["cast"].map(eval)
credits["crew"] = credits["crew"].map(eval)
credits

,cast,crew
id,,
862,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
8844,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
15602,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
31357,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
11862,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...
439050,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de..."
111109,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de..."
67758,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de..."


#### Final dataset

In [286]:
movies = movies.merge(keywords, left_index=True, right_index=True)
movies = movies.merge(credits, left_index=True, right_index=True)
movies

,adult,genres,original_language,overview,popularity,production_companies,release_date,title,vote_average,keywords,cast,crew
id,,,,,,,,,,,,
2,True,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",fi,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,"[{'name': 'Villealfa Filmproduction Oy', 'id':...",1988-10-21,Ariel,7.1,"[{'id': 240, 'name': 'underdog'}, {'id': 378, ...","[{'cast_id': 3, 'character': 'Taisto Olavi Kas...","[{'credit_id': '52fe420dc3a36847f800001f', 'de..."
3,True,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",fi,"An episode in the life of Nikander, a garbage ...",2.292110,"[{'name': 'Villealfa Filmproduction Oy', 'id':...",1986-10-16,Shadows in Paradise,7.1,"[{'id': 1361, 'name': 'salesclerk'}, {'id': 17...","[{'cast_id': 5, 'character': 'Nikander', 'cred...","[{'credit_id': '52fe420dc3a36847f8000077', 'de..."
5,True,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",en,It's Ted the Bellhop's first night on the job....,9.026586,"[{'name': 'Miramax Films', 'id': 14}, {'name':...",1995-12-09,Four Rooms,6.5,"[{'id': 612, 'name': 'hotel'}, {'id': 613, 'na...","[{'cast_id': 42, 'character': 'Ted the Bellhop...","[{'credit_id': '52fe420dc3a36847f800011b', 'de..."
6,True,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",en,"While racing to a boxing match, Frank, Mike, J...",5.538671,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",1993-10-15,Judgment Night,6.4,"[{'id': 520, 'name': 'chicago'}, {'id': 2231, ...","[{'cast_id': 7, 'character': 'Frank Wyatt', 'c...","[{'credit_id': '52fe420dc3a36847f800023d', 'de..."
11,True,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",en,Princess Leia is captured and held hostage by ...,42.149697,"[{'name': 'Lucasfilm', 'id': 1}, {'name': 'Twe...",1977-05-25,Star Wars,8.1,"[{'id': 803, 'name': 'android'}, {'id': 4270, ...","[{'cast_id': 3, 'character': 'Luke Skywalker',...","[{'credit_id': '52fe420dc3a36847f8000437', 'de..."
...,...,...,...,...,...,...,...,...,...,...,...,...
465044,True,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na...",en,A horror comedy spoofing conspiracy theory mov...,0.281008,[],2017-06-28,Abduction,0.0,[],"[{'cast_id': 0, 'character': 'Bozena', 'credit...","[{'credit_id': '595f97249251412251083338', 'de..."
467731,True,"[{'id': 18, 'name': 'Drama'}]",en,Fifteen-year-old girl Dotty Fisher is assaulte...,0.001189,[],1956-02-19,Tragedy in a Temporary Town,0.0,[],"[{'cast_id': 2, 'character': 'Alec Beggs', 'cr...","[{'credit_id': '5975903bc3a3680cb700867b', 'de..."
468343,True,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",fi,"In the 1910s, beautiful young Silja loses both...",0.001202,[],1956-01-01,Silja - nuorena nukkunut,0.0,[],[],"[{'credit_id': '597ae87a925141364e00a73e', 'de..."


In [287]:
def get_director(element):
    element = list(filter(lambda el: el["job"] == "Director", element))
    if not element:
        return ""
    else:
        return element[0]["name"]

In [288]:
def parse_to_list(element):
    l = []
    for el in element:
        l.append(el["name"])
    return l

In [289]:
def parse_cast_to_list(element):
    l = []
    for el in element[:5]:
        l.append(el["name"])
    return l

In [290]:
movies["director"] = movies["crew"].map(get_director)
movies = movies.drop(columns=["crew"])
movies.head()

,adult,genres,original_language,overview,popularity,production_companies,release_date,title,vote_average,keywords,cast,director
id,,,,,,,,,,,,
2,True,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",fi,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,"[{'name': 'Villealfa Filmproduction Oy', 'id':...",1988-10-21,Ariel,7.1,"[{'id': 240, 'name': 'underdog'}, {'id': 378, ...","[{'cast_id': 3, 'character': 'Taisto Olavi Kas...",Aki Kaurismäki
3,True,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",fi,"An episode in the life of Nikander, a garbage ...",2.292110,"[{'name': 'Villealfa Filmproduction Oy', 'id':...",1986-10-16,Shadows in Paradise,7.1,"[{'id': 1361, 'name': 'salesclerk'}, {'id': 17...","[{'cast_id': 5, 'character': 'Nikander', 'cred...",Aki Kaurismäki
5,True,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",en,It's Ted the Bellhop's first night on the job....,9.026586,"[{'name': 'Miramax Films', 'id': 14}, {'name':...",1995-12-09,Four Rooms,6.5,"[{'id': 612, 'name': 'hotel'}, {'id': 613, 'na...","[{'cast_id': 42, 'character': 'Ted the Bellhop...",Allison Anders
6,True,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",en,"While racing to a boxing match, Frank, Mike, J...",5.538671,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",1993-10-15,Judgment Night,6.4,"[{'id': 520, 'name': 'chicago'}, {'id': 2231, ...","[{'cast_id': 7, 'character': 'Frank Wyatt', 'c...",Stephen Hopkins
11,True,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",en,Princess Leia is captured and held hostage by ...,42.149697,"[{'name': 'Lucasfilm', 'id': 1}, {'name': 'Twe...",1977-05-25,Star Wars,8.1,"[{'id': 803, 'name': 'android'}, {'id': 4270, ...","[{'cast_id': 3, 'character': 'Luke Skywalker',...",George Lucas


In [292]:
movies["genres"] = movies["genres"].map(parse_to_list)
movies["production_companies"] = movies["production_companies"].map(parse_to_list)
movies["keywords"] = movies["keywords"].map(parse_to_list)
movies["cast"] = movies["cast"].map(parse_cast_to_list)
movies.head()

,adult,genres,original_language,overview,popularity,production_companies,release_date,title,vote_average,keywords,cast,director
id,,,,,,,,,,,,
2,True,"[Drama, Crime]",fi,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,"[Villealfa Filmproduction Oy, Finnish Film Fou...",1988-10-21,Ariel,7.1,"[underdog, prison, factory worker, prisoner, h...","[Turo Pajala, Susanna Haavisto, Matti Pellonpä...",Aki Kaurismäki
3,True,"[Drama, Comedy]",fi,"An episode in the life of Nikander, a garbage ...",2.292110,[Villealfa Filmproduction Oy],1986-10-16,Shadows in Paradise,7.1,"[salesclerk, helsinki, garbage, independent film]","[Matti Pellonpää, Kati Outinen, Sakari Kuosman...",Aki Kaurismäki
5,True,"[Crime, Comedy]",en,It's Ted the Bellhop's first night on the job....,9.026586,"[Miramax Films, A Band Apart]",1995-12-09,Four Rooms,6.5,"[hotel, new year's eve, witch, bet, hotel room...","[Tim Roth, Antonio Banderas, Jennifer Beals, M...",Allison Anders
6,True,"[Action, Thriller, Crime]",en,"While racing to a boxing match, Frank, Mike, J...",5.538671,"[Universal Pictures, Largo Entertainment, JVC ...",1993-10-15,Judgment Night,6.4,"[chicago, drug dealer, boxing match, escape, o...","[Emilio Estevez, Cuba Gooding Jr., Denis Leary...",Stephen Hopkins
11,True,"[Adventure, Action, Science Fiction]",en,Princess Leia is captured and held hostage by ...,42.149697,"[Lucasfilm, Twentieth Century Fox Film Corpora...",1977-05-25,Star Wars,8.1,"[android, galaxy, hermit, death star, lightsab...","[Mark Hamill, Harrison Ford, Carrie Fisher, Pe...",George Lucas


### Recommendator

In [293]:
movies.head()

,adult,genres,original_language,overview,popularity,production_companies,release_date,title,vote_average,keywords,cast,director
id,,,,,,,,,,,,
2,True,"[Drama, Crime]",fi,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,"[Villealfa Filmproduction Oy, Finnish Film Fou...",1988-10-21,Ariel,7.1,"[underdog, prison, factory worker, prisoner, h...","[Turo Pajala, Susanna Haavisto, Matti Pellonpä...",Aki Kaurismäki
3,True,"[Drama, Comedy]",fi,"An episode in the life of Nikander, a garbage ...",2.292110,[Villealfa Filmproduction Oy],1986-10-16,Shadows in Paradise,7.1,"[salesclerk, helsinki, garbage, independent film]","[Matti Pellonpää, Kati Outinen, Sakari Kuosman...",Aki Kaurismäki
5,True,"[Crime, Comedy]",en,It's Ted the Bellhop's first night on the job....,9.026586,"[Miramax Films, A Band Apart]",1995-12-09,Four Rooms,6.5,"[hotel, new year's eve, witch, bet, hotel room...","[Tim Roth, Antonio Banderas, Jennifer Beals, M...",Allison Anders
6,True,"[Action, Thriller, Crime]",en,"While racing to a boxing match, Frank, Mike, J...",5.538671,"[Universal Pictures, Largo Entertainment, JVC ...",1993-10-15,Judgment Night,6.4,"[chicago, drug dealer, boxing match, escape, o...","[Emilio Estevez, Cuba Gooding Jr., Denis Leary...",Stephen Hopkins
11,True,"[Adventure, Action, Science Fiction]",en,Princess Leia is captured and held hostage by ...,42.149697,"[Lucasfilm, Twentieth Century Fox Film Corpora...",1977-05-25,Star Wars,8.1,"[android, galaxy, hermit, death star, lightsab...","[Mark Hamill, Harrison Ford, Carrie Fisher, Pe...",George Lucas


In [294]:
def filter_by_name(df: pd.DataFrame, name: str):
    for word in name.split():
        df = df[df["title"].str.contains(rf"\b{word}\b", flags=re.IGNORECASE)]
    return df

In [298]:
def get_similarity(movie: pd.DataFrame):
    def similarity(movie_to_compare: pd.DataFrame):
        score = 0
        score += 0.2 if movie["adult"] == movie_to_compare["adult"] else 0
        score += 0.4*len(set(movie["genres"]) & set(movie_to_compare["genres"]))
        score += 0.25*len(set(movie["production_companies"]) & set(movie_to_compare["production_companies"]))
        score += 0.4*len(set(movie["keywords"]) & set(movie_to_compare["keywords"]))
        score += 0.25*len(set(movie["cast"]) & set(movie_to_compare["cast"]))
        score += 0.5 if movie["director"] == movie_to_compare["director"] else 0
        score += movie["popularity"]*movie["vote_average"]/4000
        score += 0.5 if abs(movie["release_date"].year - movie_to_compare["release_date"].year) < 10 else 0
        return score

In [295]:
filter_by_name(movies, "star wars")

,adult,genres,original_language,overview,popularity,production_companies,release_date,title,vote_average,keywords,cast,director
id,,,,,,,,,,,,
11,True,"[Adventure, Action, Science Fiction]",en,Princess Leia is captured and held hostage by ...,42.149697,"[Lucasfilm, Twentieth Century Fox Film Corpora...",1977-05-25,Star Wars,8.1,"[android, galaxy, hermit, death star, lightsab...","[Mark Hamill, Harrison Ford, Carrie Fisher, Pe...",George Lucas
1893,True,"[Adventure, Action, Science Fiction]",en,"Anakin Skywalker, a young slave strong with th...",15.649091,[Lucasfilm],1999-05-19,Star Wars: Episode I - The Phantom Menace,6.4,"[prophecy, senate, queen, taskmaster, galaxy, ...","[Liam Neeson, Ewan McGregor, Natalie Portman, ...",George Lucas
1894,True,"[Adventure, Action, Science Fiction]",en,"Ten years after the invasion of Naboo, the gal...",14.072511,[Lucasfilm],2002-05-15,Star Wars: Episode II - Attack of the Clones,6.4,"[senate, investigation, army, death star, jedi...","[Ewan McGregor, Natalie Portman, Hayden Christ...",George Lucas
1895,True,"[Science Fiction, Adventure, Action]",en,"Years after the onset of the Clone Wars, the n...",13.165421,[Lucasfilm],2005-05-17,Star Wars: Episode III - Revenge of the Sith,7.1,"[showdown, death star, vision, cult figure, ha...","[Ewan McGregor, Natalie Portman, Hayden Christ...",George Lucas
12180,True,"[Thriller, Animation, Action, Science Fiction,...",en,Set between Episode II and III the Clone Wars ...,8.524857,[Lucasfilm],2008-08-05,Star Wars: The Clone Wars,5.8,"[android, jedi, sith, 3d, clone army, separati...","[Matt Lanter, Ashley Eckstein, James Arnold Ta...",Dave Filoni
42979,True,"[Animation, Comedy, Science Fiction]",en,"Fans of Adult Swim's ""Robot Chicken"" and the S...",7.994542,[Lucasfilm],2007-07-17,Robot Chicken: Star Wars,6.9,[],"[Candace Bailey, Bob Bergen, Ahmed Best, Seth ...",Seth Green
74849,True,"[Adventure, Comedy, Family, Science Fiction, T...",en,Luke Skywalker and Han Solo battle evil Imperi...,4.099014,"[20th Century Fox Television, Smith-Hemion Pro...",1978-12-01,The Star Wars Holiday Special,3.2,"[holiday, musical, space, jedi, drunk, space o...","[Harrison Ford, Mark Hamill, Anthony Daniels, ...",Steve Binder
76180,True,[Documentary],en,Empire of Dreams: The Story of the Star Wars T...,5.796763,"[Lucasfilm, Fox Television Studios, Prometheus...",2004-09-12,Empire of Dreams: The Story of the Star Wars T...,7.1,"[film history, audition, making of, space opera]","[Robert Clotworthy, Kenny Baker, Jim Bloom, Le...",Kevin Burns
140607,True,"[Action, Adventure, Science Fiction, Fantasy]",en,Thirty years after defeating the Galactic Empi...,31.626013,"[Lucasfilm, Truenorth Productions, Bad Robot]",2015-12-15,Star Wars: The Force Awakens,7.5,"[android, spaceship, jedi, imax, space opera, 3d]","[Daisy Ridley, John Boyega, Adam Driver, Harri...",J.J. Abrams
